In [6]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
# Import the API key.
from config import g_key

In [35]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")

#Create and initialize Y/N columns for rain or snow
city_data_df.loc[city_data_df['Rain (inches)'] > 0,'rain_flg'] = 'Y'
city_data_df.loc[city_data_df['Rain (inches)'] == 0,'rain_flg'] = 'N'       
city_data_df.loc[city_data_df['Snow (inches)'] > 0,'snow_flg'] = 'Y'
city_data_df.loc[city_data_df['Snow (inches)'] == 0,'snow_flg'] = 'N'     

city_data_df


# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    


In [31]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? \n"))
max_temp = float(input("What is the maximum temperature you would like for your trip? \n"))
rain_input_flg = str(input("Do you want it to be raining? (yes/no) \n"))
snow_input_flg = str(input("Do you want it to be snowing? (yes/no) \n"))

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


if rain_input_flg == "yes":
    preferred_cities_df = preferred_cities_df.loc[preferred_cities_df['rain_flg'] == 'Y']
    
if snow_input_flg == "yes":
    preferred_cities_df = preferred_cities_df.loc[preferred_cities_df['snow_flg'] == 'Y'] 

preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip? 
10
What is the maximum temperature you would like for your trip? 
100
Do you want it to be raining? (yes/no) 
yes
Do you want it to be snowing? (yes/no) 
no


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),rain_flg,snow_flg
18,18,Hobart,AU,-42.88,147.33,53.60,93,75,5.82,light intensity shower rain,1.61,0,Y,N
28,28,Ko Samui,TH,9.54,99.94,83.46,76,100,11.48,moderate rain,6.59,0,Y,N
33,33,Shimoda,JP,34.67,138.95,63.00,95,100,7.00,heavy intensity rain,10.35,0,Y,N
39,39,Hamilton,US,39.18,-84.53,75.00,78,90,4.72,light rain,1.73,0,Y,N
49,49,Hirara,JP,24.80,125.28,83.64,85,100,13.98,light rain,5.49,0,Y,N
57,57,Albany,US,42.60,-73.97,91.99,39,52,1.01,light rain,1.08,0,Y,N
66,66,Alad,PH,12.62,122.25,83.82,77,100,11.05,light rain,2.90,0,Y,N
68,68,Birao,CF,10.28,22.79,77.02,72,99,6.06,moderate rain,0.51,0,Y,N
69,69,Bonthe,SL,7.53,-12.51,79.50,85,100,6.96,light rain,9.17,0,Y,N
72,72,Amapa,BR,1.00,-52.00,76.78,95,100,3.38,moderate rain,6.81,0,Y,N


In [32]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country","Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
18,Hobart,AU,light intensity shower rain,53.60,-42.88,147.33,
28,Ko Samui,TH,moderate rain,83.46,9.54,99.94,
33,Shimoda,JP,heavy intensity rain,63.00,34.67,138.95,
39,Hamilton,US,light rain,75.00,39.18,-84.53,
49,Hirara,JP,light rain,83.64,24.80,125.28,
57,Albany,US,light rain,91.99,42.60,-73.97,
66,Alad,PH,light rain,83.82,12.62,122.25,
68,Birao,CF,moderate rain,77.02,10.28,22.79,
69,Bonthe,SL,light rain,79.50,7.53,-12.51,
72,Amapa,BR,moderate rain,76.78,1.00,-52.00,


In [33]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
hotel_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
18,Hobart,AU,light intensity shower rain,53.60,-42.88,147.33,St Ives Apartments
28,Ko Samui,TH,moderate rain,83.46,9.54,99.94,"The Passage Samui Villas & Resort, Thailand"
33,Shimoda,JP,heavy intensity rain,63.00,34.67,138.95,Shimoda Tokyu Hotel
39,Hamilton,US,light rain,75.00,39.18,-84.53,Six Acres Bed & Breakfast
49,Hirara,JP,light rain,83.64,24.80,125.28,Hotel Atoll Emerald Miyakojima
...,...,...,...,...,...,...,...
526,The Pas,CA,light rain,59.00,53.83,-101.25,Wescana Inn
532,Rapid Valley,US,light rain,77.00,44.06,-103.15,Fairfield Inn & Suites by Marriott Rapid City
538,Champerico,GT,light rain,85.98,14.30,-91.92,"Hotel y restaurante ""El Diamante"""
540,Corn Island,NI,moderate rain,80.89,12.17,-83.04,Big Fish Guest House


In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")